In [1]:
import os
base=os.path.dirname(os.path.realpath("__file__")).split(os.sep)
os.chdir(os.sep+os.path.join(*base[:-1]))
os.getcwd()

'/Users/haihabi/projects/NormFlowPy'

In [2]:
import numpy as np
import normflowpy as nfp
import torch
from matplotlib import pyplot as plt
from torch.distributions import MultivariateNormal
from experiments.functions import run_training
from torchvision import  transforms
import torchvision

/Users/haihabi/envs/research_3_7/lib/python3.7/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/Users/haihabi/envs/research_3_7/lib/python3.7/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: /Users/haihabi/envs/research_3_7/lib/python3.7/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 14.0.0 or later, but libjpeg.9.dylib provides version 12.0.0
  warn(f"Failed to load image Python extension: {e}")


In [3]:
n_flow_blocks = 3
batch_size = 32
n_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Current Working Device is set to:" + str(device))
transform=transforms.Compose([
        transforms.ToTensor()
        ])

def collater(batch):
    return torch.stack([b[0] for b in batch],dim=0)




training_data=torchvision.datasets.MNIST('data', train=True, download=True,
                       transform=transform)
training_dataset_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size,
                                                      shuffle=True, num_workers=0,collate_fn=collater)
validation_data=torchvision.datasets.MNIST('data', train=False, download=True,
                       transform=transform)
validation_dataset_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size,
                                                        shuffle=False, num_workers=0,collate_fn=collater)

Current Working Device is set to:cpu


# Create Glow Normalizing Flow Model

In [4]:
x_shape=[1,28,28]
dim=np.prod(x_shape)
base_distribution = MultivariateNormal(torch.zeros(dim, device=device),
                                       torch.eye(dim, device=device))  # generate a class for base distribution
flows = []
for s in range(2):
    x_shape=[x_shape[0]*4,x_shape[1]//2,x_shape[2]//2]
    flows.append(nfp.flows.Squeeze(2))
    for i in range(n_flow_blocks):
        flows.append(
            nfp.flows.ActNorm(x_shape=x_shape))
        flows.append(
            nfp.flows.InvertibleConv2d1x1(dim=x_shape[0]))
        flows.append(
            nfp.flows.AffineCoupling(x_shape=x_shape, parity=i % 2,  nh=32))
flows.append(nfp.flows.Tensor2Vector(x_shape))
flow = nfp.NormalizingFlowModel(base_distribution, flows).to(device)

# Set Optimizer and run training

In [ ]:
optimizer = torch.optim.Adam(flow.parameters(), lr=1e-3,weight_decay=1e-4)
run_training(n_epochs, training_dataset_loader, validation_dataset_loader, flow, optimizer, device)

Starting Training Loop


End Epoch with training loss:332841.7468868862 and validtion loss:14617.314184560562
End Epoch with training loss:9404.86828326823 and validtion loss:3781.7692498399047
End Epoch with training loss:3065.1074485026043 and validtion loss:6415.190694181302
End Epoch with training loss:1187.0502162760417 and validtion loss:24800.822864844777


# Plot Samples